In [147]:
import pickle
import pandas as pd
import sqlite3
import os
import numpy as np

In [11]:
PATH_pred = os.path.join("predictions")
Year = 2019
Quarter = 1

# Equal weight

In [70]:
def retrieve_stock_return(co_id_list, Year, Quarter):
    ''' Given company list, return each (absolute) stock return '''
    dbName = 'stock.db'
    db_path = os.path.join('..', 'Database', dbName)
    connection = sqlite3.connect(db_path)
    cursor = connection.cursor()

    return_list = list()

    for co_id in co_id_list:
        query = f'SELECT "Stock Return" FROM t_final_data WHERE CO_ID="{co_id}" and Year={Year} and Quarter={Quarter}'
        return_list.append(cursor.execute(query).fetchone()[0])

    connection.close()

    return return_list

In [25]:
def gen_equal_weight_return(Year, Quarter, Model):
    '''
        write into t_return_equal_weight @ portfolio_return.db
    '''
    strategy="equal weight"

    RETURN_TABLE = 't_return_equal_weight'
    dbName = 'portfolio_return.db'
    db_path = os.path.join('..', 'Database', dbName)
    connection = sqlite3.connect(db_path)
    cursor = connection.cursor()


    with open(os.path.join(PATH_pred, Model, f"{Year}-{Quarter}-company-rank.pickle"), "rb") as fp:
        co_id_rank = pickle.load(fp)

    # select top 25 companies
    co_id_list = co_id_rank[:25]

    # collect (absolute) stock return from t_final_data @ stock.db
    return_list = retrieve_stock_return(co_id_list, Year, Quarter)

    # equal weight
    stock_return = np.mean(return_list)

    # overwrite existing row or insert a new row (sqlite)
    if cursor.execute(f'SELECT * FROM {RETURN_TABLE} WHERE Model = "{Model}" AND Strategy = "{strategy}" AND Year = {Year} AND Quarter = {Quarter}').fetchone() is None:
        data = Model, strategy, Year, Quarter, stock_return
        query = f'INSERT INTO {RETURN_TABLE} (Model, Strategy, Year, Quarter, "Stock Return") VALUES (?,?,?,?,?);'
        print('[QUERY] ' + query) 
        print('[DATA] ' + data) 
        cursor.execute(query, data)
    else:
        query = f'UPDATE {RETURN_TABLE} \
            SET "Stock Return" = {stock_return} \
            WHERE \
                Model = "{Model}" AND Strategy = "{strategy}" AND Year = {Year} AND Quarter = {Quarter} \
            '
        print('[QUERY] ' + query) 
        cursor.execute(query)

    connection.commit()
    connection.close()

In [ ]:
def gen_portfolio_return(Year, Quarter, Model, Strategy):
    if Strategy == "equal weight":
        gen_equal_weight_return(Year, Quarter, Model)
    else:
        raise Exception("[ERROR] we don't have this strategies yet...")

In [2]:
np.mean(return_list)

NameError: name 'np' is not defined

In [103]:
from datetime import date, datetime

today = date.today()
year, month, day = today.strftime("%Y/%m/%d").split('/')

In [130]:
def date_to_quarter(month, day):
    ''' according to our definition of each season's timestamps '''
    month = int(month)
    day = int(day)

    if month < 2 and day < 26:
        quarter = 3
    elif month < 5 and day < 26:
        quarter = 4
    elif month < 8 and day < 26:
        quarter = 1
    elif month < 11 and day < 26:
        quarter = 2
    else:
        quarter = 3

    return str(quarter)

In [145]:
date_to_quarter(9, 13)

'2'

In [135]:
date(2022, 11,13).strftime("%Y")

'2022'

In [ ]:
Q1_START = ('-05-20', '-05-25')
Q1_END = ('-08-19', '-08-24')
Q2_START = ('-08-20', '-08-25')
Q2_END = ('-11-19', '-11-24')
Q3_START = ('-11-20', '-11-25')
Q3_END = ('-02-19', '-02-27')
Q4_START = ('-02-20', '-02-27')
Q4_END = ('-05-19', '-05-24')
5, 20
8, 20
11, 20
2, 20

In [1]:
import yfinance as yf
df = yf.download("2330.TW", start="2022-11-30", end="2026-12-31")
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-11-30,483.0,490.0,482.5,490.0,487.195648,57111603
2022-12-01,506.0,508.0,498.5,498.5,495.647003,41746878
2022-12-02,490.0,497.0,490.0,492.5,489.681335,28319679
2022-12-05,491.5,497.5,489.0,489.0,486.201355,28360444
2022-12-06,488.0,489.0,478.0,478.0,475.264313,40561068
2022-12-07,477.0,485.5,475.0,475.0,472.281494,36563503
2022-12-08,475.0,475.0,467.0,471.5,468.801514,27238567
2022-12-09,478.0,484.5,476.0,481.5,478.744293,25915337
2022-12-12,476.5,478.0,473.5,475.0,472.281494,20973334
